In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

from qubex.simulator import Control, QuantumSimulator, QuantumSystem, Transmon

In [ ]:
transmon = Transmon(
    label="Q00",
    dimension=3,
    frequency=7.5,
    anharmonicity=-0.35,
)

system = QuantumSystem(objects=[transmon])

simulator = QuantumSimulator(system)

initial_state = system.state({transmon.label: "0"})

In [ ]:
from qubex.pulse import Drag

duration = 16

alpha = 2 * np.pi * transmon.anharmonicity

pulse = Drag(
    duration=duration,
    amplitude=1,
    beta=-0.5 / alpha,
)
norm_factor = np.pi / float(np.sum(np.abs(pulse.values) * pulse.SAMPLING_PERIOD))
pulse = pulse.scaled(norm_factor)
pulse.plot()

In [ ]:
Control.SAMPLING_PERIOD = Drag.SAMPLING_PERIOD

control = Control(
    target=transmon.label,
    frequency=transmon.frequency,
    waveform=pulse.values,
)

In [ ]:
control.plot()

In [ ]:
N = control.length
F_ = np.fft.fft(control.waveform)
f_ = np.fft.fftfreq(N, control.SAMPLING_PERIOD)
F = np.fft.fftshift(F_)
f = np.fft.fftshift(f_)

plt.plot(f, F.real, label="real")
plt.plot(f, F.imag, label="imag")
plt.plot(f, np.abs(F), label="abs")
plt.grid()
plt.legend()

In [ ]:
result = simulator.simulate(
    controls=[control],
    initial_state=initial_state,
)

In [ ]:
result.show_last_population(transmon.label)

In [ ]:
result.plot_population_dynamics()

In [ ]:
result.display_bloch_sphere(transmon.label)